In [ ]:
%pip install setfit

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [4]:
!git config --global user.email "ja_mahmahi@esi.dz"
!git config --global user.name "Anis Mahmahi"

In [5]:
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [53]:
import pandas as pd

train = pd.read_csv("/content/train-multi-label-oana.csv")
test = pd.read_csv("/content/test-multi-label-oana.csv")

In [54]:
train['Flag-Waving'].value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.0    3920
1.0     227
Name: Flag-Waving, dtype: int64

In [55]:
hq = pd.read_csv("/content/df_tweets_HiQualProp.csv")
hq

,id,text,labels,strategy,author_id,labels_weak3,labels_weak2,labels_weak1,propaganda_category,text_normalized
0,1372969669389393932,I only see a country corrupted and economicall...,0,general discourse,1231542469252255745,0,0,0,none,I only see a country corrupted and economicall...
1,1521594471279996928,New to you that Pope is on the side of murder ...,0,general discourse,1231542469252255745,0,0,0,none,New to you that Pope is on the side of murder ...
2,1385896329071632386,BJP PROPAGANDA AGAINST DELHI GOVT EXPOSED Tot...,0,general discourse,2991581923,1,0,0,none,BJP PROPAGANDA AGAINST DELHI GOVT EXPOSED Tota...
3,1403386698835365888,"Senate Democrats threaten to subpoena Barr, Se...",0,general discourse,2467791,1,0,0,none,"Senate Democrats threaten to subpoena Barr , S..."
4,1509240723362795531,Analysis: Russia understands Trump's utility e...,0,general discourse,2467791,1,0,0,none,Analysis : Russia understands Trump 's utility...
...,...,...,...,...,...,...,...,...,...,...
29591,1564600447897243648,If your response to losing an election is to t...,0,general discourse,1187465769778442240,1,0,0,none,If your response to losing an election is to t...
29592,1569421814798876673,@brakerbaker @chrissyclark_ Biden is the one f...,1,against Western countries,1348870080856252417,1,1,1,name_calling_labelling,@USER @USER Biden is the one funding fascists ...
29593,1570758041938841602,The historic victory of Ukraine in the Kharkiv...,0,general discourse,2152685546,0,0,0,none,The historic victory of Ukraine in the Kharkiv...
29594,1573712319078871045,You can’t speak out about the injustice done t...,0,general discourse,1556105145716740096,1,0,0,none,You can't speak out about the injustice done t...


In [56]:
hq['propaganda_category'].value_counts()

none                           25559
slogans                         1584
name_calling_labelling          1071
loaded_language                  734
appeal_to_fear_prejudice         492
doubt                             64
reductio_ad_hitelerium            50
bandwagon                         25
flag_waving                       13
black_and_white_fallacy            1
appeal_to_authority                1
thought_terminating_cliches        1
whataboutism                       1
Name: propaganda_category, dtype: int64

In [58]:
hq_cleaned = hq[hq['propaganda_category']=='flag_waving'][['text_normalized', 'propaganda_category']]
hq_cleaned

,text_normalized,propaganda_category
1122,It is kind of sad that the reality Ukrainian n...,flag_waving
3415,":lying_face: Russia claims that "" Ukrainian na...",flag_waving
4317,"@USER @USER > "" The Ukrainian army have been p...",flag_waving
5279,@USER @USER I would say the Ukrainian National...,flag_waving
5639,The car bomb that killed hardline TV commentat...,flag_waving
6060,UKRAINE WAS PART OF RUSSIA ! From the same Eth...,flag_waving
10747,@USER @USER @USER but no reports of mass killi...,flag_waving
10898,"And , by the way , BOTH OF MY GRANDFATHERS HAD...",flag_waving
12379,@USER @USER @USER Are you aware that pro-Russi...,flag_waving
13065,:warning: Ukrainian nationalists shell #Novovo...,flag_waving


In [59]:
hq_cleaned.rename(columns={'text_normalized':'text', 'propaganda_category':'label'}, inplace=True)

In [60]:
hq_cleaned['label']=1


In [61]:
hq_cleaned

,text,label
1122,It is kind of sad that the reality Ukrainian n...,1
3415,":lying_face: Russia claims that "" Ukrainian na...",1
4317,"@USER @USER > "" The Ukrainian army have been p...",1
5279,@USER @USER I would say the Ukrainian National...,1
5639,The car bomb that killed hardline TV commentat...,1
6060,UKRAINE WAS PART OF RUSSIA ! From the same Eth...,1
10747,@USER @USER @USER but no reports of mass killi...,1
10898,"And , by the way , BOTH OF MY GRANDFATHERS HAD...",1
12379,@USER @USER @USER Are you aware that pro-Russi...,1
13065,:warning: Ukrainian nationalists shell #Novovo...,1


In [62]:
train_cleaned = train[['text','Flag-Waving']].rename(columns={'Flag-Waving':"label"})
train_cleaned

,text,label
0,Pamela Geller and Robert Spencer co-founded an...,0.0
1,"He added: ""We condemn all those whose behaviou...",0.0
2,"Ms Geller, of the Atlas Shrugs blog, and Mr Sp...",0.0
3,On both of their blogs the pair called their b...,0.0
4,The UK should never become a stage for inflamm...,1.0
...,...,...
4142,"Comrades, these jokes you have been listening ...",0.0
4143,In the future everyone will be the subject of ...,0.0
4144,Also the Left killed comedy.\n,0.0
4145,That's what Columbia snowflakes thought was of...,0.0


In [63]:
test_cleaned = test[['text','Flag-Waving']].rename(columns={'Flag-Waving':"label"})
test_cleaned

,text,label
0,"""The next transmission could be more pronounce...",0
1,This along with poor sanitation and vector con...,0
2,Geneva - The World Health Organisation chief o...,0
3,"But Tedros voiced alarm that ""plague in Madaga...",0
4,He also pointed to the presence of the pneumon...,0
...,...,...
982,The U.S. Department of Justice said the court ...,0
983,The Acosta-CNN lawsuit was filed by Ted Boutro...,0
984,"The yanking of the physical credential, by the...",0
985,"The president described Acosta as ""unbelievabl...",0


In [64]:
result_df = pd.concat([train_cleaned, hq_cleaned], ignore_index=True)
result_df

,text,label
0,Pamela Geller and Robert Spencer co-founded an...,0.0
1,"He added: ""We condemn all those whose behaviou...",0.0
2,"Ms Geller, of the Atlas Shrugs blog, and Mr Sp...",0.0
3,On both of their blogs the pair called their b...,0.0
4,The UK should never become a stage for inflamm...,1.0
...,...,...
4155,@USER @USER @USER Are you aware that pro-Russi...,1.0
4156,:warning: Ukrainian nationalists shell #Novovo...,1.0
4157,"@USER 14,000 dead ethnic Russians in the Donba...",1.0
4158,Ukrainian Nationalist Volunteers Committing ' ...,1.0


In [65]:
result_df = result_df.dropna(subset=['text'])

In [66]:
result_df = result_df.sample(frac=1).reset_index(drop=True)

In [67]:
result_df.to_csv("train_cleaned.csv",index=False)

In [68]:
test_cleaned.to_csv("test_cleaned.csv",index=False)

In [69]:
from datasets import load_dataset


dataset = load_dataset('csv', data_files={'train': "/content/train_cleaned.csv",'test': "/content/test_cleaned.csv"}, )
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4159
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 987
    })
})

In [70]:
from setfit import SetFitModel
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression

model_body=SentenceTransformer("sentence-transformers/paraphrase-mpnet-base-v2")
model_head=LogisticRegression(class_weight="balanced")
model = SetFitModel(model_body=model_body, model_head=model_head,labels=[0,1],)


# # Load a SetFit model from Hub
# model = SetFitModel.from_pretrained(
#     "sentence-transformers/paraphrase-mpnet-base-v2",
#     labels=[0,1],
# )


In [71]:
model.to("cuda")

SetFitModel(model_body=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
), model_head=LogisticRegression(class_weight='balanced'), multi_target_strategy=None, normalize_embeddings=False, labels=[0, 1], model_card_data=SetFitModelCardData(language=None, license=None, tags=['setfit', 'sentence-transformers', 'text-classification', 'generated_from_setfit_trainer'], model_name='SetFit', model_id=None, dataset_name=None, dataset_id=None, dataset_revision=None, task_name=None, st_id=None, hyperparameters={}, eval_results_dict={}, eval_lines_list=[], metric_lines=[], widget=[], predict_example=None, label_example_list=[], tokenizer_warning=False, train_set_metrics_list=[], train_set_sentences_per_label_list=[], code_carbon_cal

In [72]:
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset

args = TrainingArguments(
    batch_size=16,
    num_epochs=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    num_iterations=5,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    metric="f1",
    column_mapping={"text": "text", "label": "label"}  # Map dataset columns to text/label expected by trainer
)


Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/4159 [00:00<?, ? examples/s]

In [73]:
import torch
training_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
training_device

device(type='cuda')

In [74]:
# Train and evaluate
trainer.train()


***** Running training *****
  Num examples = 2600
  Num epochs = 2
  Total optimization steps = 5200
  Total train batch size = 16


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Embedding Loss,Rate
1,No log,No log,0.187500,0.000011
2,No log,No log,0.139300,0.000000


  0%|          | 0/617 [00:00<?, ?it/s]

  0%|          | 0/617 [00:00<?, ?it/s]

Loading best SentenceTransformer model from step 5200.


In [78]:
metrics = trainer.evaluate(dataset['test'])
print(metrics)


Applying column mapping to the evaluation dataset
***** Running evaluation *****


{'f1': 0.7514450867052023}


In [76]:
trainer.push_to_hub("anismahmahi/G1-setfit-model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/6.97k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/anismahmahi/G1-setfit-model/commit/db3cce7a0386bd6f185149947318f6e4b51fe524', commit_message='Add SetFit model', commit_description='', oid='db3cce7a0386bd6f185149947318f6e4b51fe524', pr_url=None, pr_revision=None, pr_num=None)

In [81]:
from sklearn.metrics import f1_score,classification_report
preds = model.predict(list(test['text']))



In [83]:
print(classification_report(dataset['test']['label'],preds))

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.98       902
         1.0       0.74      0.76      0.75        85

    accuracy                           0.96       987
   macro avg       0.86      0.87      0.86       987
weighted avg       0.96      0.96      0.96       987

